In [ ]:
# Standard imports. Note: You must pip install nasdaqdatalink 1st
import os
import pandas as pd
import hvplot.pandas
from pathlib import Path
import requests

# For API Calls
import nasdaqdatalink
# For opening zip folder
import shutil 
# For technical analysis
import pandas_ta as ta

from MCForecastTools import MCSimulation

In [ ]:
# Linking my API key to .env in the same folder. The key is stored in the folder without any quotations around it 

nasdaqdatalink.read_key(filename=".env")

In [ ]:
# A function to retrieve a dataframe of counties, zips, etc
def get_regions(regions):
    region_df=nasdaqdatalink.get_table('ZILLOW/REGIONS', region_type=regions)  
    return region_df

# 1. Get the regions data from Zillow REST APIs.   
This contains a list of all counties in the US.

In [ ]:
# Using get_regions to retrieve a list of counties
region_df = get_regions('county')
region_df[["county", "state"]] = region_df["region"].str.split(';', 1, expand=True)
region_df["state"] = region_df["state"].str.split(';', 1, expand=True)[0]

#
# Clean up regions data
# Remove ' County' so that we can match the Zillow data with Wikipedia data.
region_df["county"] = region_df["county"].str.replace(" County", "")

# Remove the leading blank space from the 'state' column.
region_df["state"] = region_df['state'].str[1:]

# Clean up region_id datatype.
region_df['region_id']=region_df['region_id'].astype(int)

# Check data for region_df
display(region_df.head())
display(region_df.tail())

# 2. Get the Zillow sales data.  
In this example, we read in Zillow sales data in the form of a CSV file.  

In [ ]:
# Get the Zillow sales data. 
# The actual API call using the SDK.
# Instructions can be found here https://data.nasdaq.com/databases/ZILLOW/usage/quickstart/python
# Replace 'quandl' w/ 'nasdaqdatalink
# Turned into a function to prevent constant re-downloading massive csv

def get_zillow_data():
    data = nasdaqdatalink.export_table('ZILLOW/DATA', indicator_id='ZSFH', region_id=list(region_df['region_id']),filename='db.zip')
    
    # Unzipping database from API call
    shutil.unpack_archive('db.zip')
    return data        

In [ ]:
# Reading in Database
zillow_data=pd.read_csv(
    Path('ZILLOW_DATA_d5d2ff90eb7172dbde848ea36de12dfe.csv')
)

# Check the Zillow sales data
display(zillow_data.head())
display(zillow_data.tail())

In [ ]:
## Merge the Region dataframe with the Zillow sales data
zillow_merge_df = pd.merge(region_df, zillow_data, on=['region_id'])

# Check the merged Zillow data
zillow_merge_df.head()

# 3. Get the county coordinates data.
We couldn't find the county coordinates from Zillow, so we sourced the data from Wikipedia.   We are going to have to merge the data with Zillow based on county and state. 

In [ ]:
# Read in county data with coordinates
county_coordinates_df=pd.read_csv(
    Path('counties_w_coordinates.csv')
)

# Clean up data.
# We need to rename the columns so that we can merge our Zillow data set 
# with the county coordinates data.   The dataframes will be merged against 'county' and 'state'. 
county_coordinates_df = county_coordinates_df.rename(columns={"County\xa0[2]" : "county"})
# county_coordinates_df = county_coordinates_df.rename(columns={"region" : "region"})
county_coordinates_df = county_coordinates_df.rename(columns={"State" : "state"})

# Remove degrees 
county_coordinates_df["Latitude"] = county_coordinates_df["Latitude"].str.replace("°", "")
county_coordinates_df["Longitude"] = county_coordinates_df["Longitude"].str.replace("°", "")

# Remove + sign for Latitude and Longitude
county_coordinates_df["Latitude"] = county_coordinates_df["Latitude"].str.replace("+", "")
county_coordinates_df["Longitude"] = county_coordinates_df["Longitude"].str.replace("+", "")

# Some of the data uses unicode hyphens which causes problems when trying to convert the Longitude and Latitude to float.
county_coordinates_df["Latitude"] = county_coordinates_df["Latitude"].str.replace('\U00002013', '-')
county_coordinates_df["Longitude"] = county_coordinates_df["Longitude"].str.replace('\U00002013', '-')

# Convert Longitude and Latitude to float so we can display on the map. 
county_coordinates_df["Latitude"] = county_coordinates_df["Latitude"].astype(float)
county_coordinates_df["Longitude"] = county_coordinates_df["Longitude"].astype(float)

# Check the county coordinates data
county_coordinates_df.head()

In [ ]:
# Merge the Zillow data and county coordinates data.
master_df = pd.merge(zillow_merge_df, county_coordinates_df, on=['county', 'state'])

master_df['date']=pd.to_datetime(master_df['date'])

# Check the master data
master_df

# 4. Display in a Map

In [ ]:
# Get mean data by state and county
county_df = master_df.groupby(["state", "county"]).mean()

# Divide price by 1000 so that it looks better on map.
county_df["value"] = county_df["value"] / 1000

# Check data
display(county_df.head())
display(county_df.tail())

In [ ]:
county_df.hvplot.points(
    'Longitude',
    'Latitude',
    geo=True,
    size='value',
    color='value',
    tiles='OSM',
    height=700,
    width=1200)

# The MAC/D

In [ ]:
# Creates a DataFrame using only the columns we are interested in

filtered_df = master_df[['date','county','state','value']]

filtered_df['county'] = filtered_df['county'] + ", " + filtered_df['state']
drop_cols = ['state']
filtered_df = filtered_df.drop(columns=drop_cols) 

In [ ]:
# Figured out the change in number of counties was messing up the charts

exploratory_df=filtered_df.groupby('date').count()

In [ ]:
# Create new DataFrame with summed county markets to represent the entire nation
nationwide_df = filtered_df.groupby(filtered_df['date']).agg({'value':'sum'})

# Must divide 'values' by number of counties that make up said value so data isn't skewed by county number
nationwide_df['avg'] = nationwide_df['value']/exploratory_df['county']

In [ ]:
# Define a function for getting a nationwide MACD indicator using pandas_ta
def get_nationwide_macd(fast, slow, signal):
    nationwide_macd_df = nationwide_df.ta.macd(close='avg', fast=fast, slow=slow, signal=signal, append=True)
    # Making DataFrame look nice
    nationwide_macd_df = nationwide_macd_df.rename(columns={f'MACD_{fast}_{slow}_{signal}':'fast_ema',f'MACDh_{fast}_{slow}_{signal}':'signal',f'MACDs_{fast}_{slow}_{signal}':'slow_ema'}).dropna()
    # Divide by 1000 so it looks more like a momentum indicator
    nationwide_macd_df = nationwide_macd_df/1000
    return nationwide_macd_df

In [ ]:
# Use newly defined funtion

nationwide_macd_df = get_nationwide_macd(6, 12, 4)

In [ ]:
# Graph

nationwide_macd_df.hvplot(title='US Housing Market Momentum', ylabel='Momentum')

In [ ]:
# Show mean housing price in county

filtered_df.hvplot(title='Mean Single Famiy Home Price',groupby='county', x='date', yformatter='%.0f')

In [ ]:
# Define a function for getting a county-specific MACD indicator using pandas_ta
def get_county_macd(fast, slow, signal):
    
    county_macd_df=filtered_df.copy()
    
    county_macd_df.ta.macd(close='value', fast=fast, slow=slow, signal=signal, append=True)
    
    # Making DataFrame look nice
    county_macd_df = county_macd_df.rename(columns={f'MACD_{fast}_{slow}_{signal}':'fast_ema',f'MACDh_{fast}_{slow}_{signal}':'signal',f'MACDs_{fast}_{slow}_{signal}':'slow_ema'}).dropna()
    
    county_macd_df = county_macd_df.drop(columns='value').set_index('date')

    county_macd_df[['fast_ema','signal','slow_ema']] = county_macd_df[['fast_ema','signal','slow_ema']]/1000
    
    return county_macd_df

In [ ]:
# Use newly defined function
county_macd_df=get_county_macd(6,12,4)

In [ ]:
county_macd_df.hvplot(title='MAC/D by County', groupby='county', x='date', ylabel='Momentum')

# Monte Carlo

In [ ]:
filtered_df = master_df[['date','county','state','value']]
filtered_df = filtered_df.sort_values('value', ascending=False)

In [ ]:
# Deleting county and state columns and replacing with location column which contains county, state
# This is necessary because same county names exist in different states. 
mc_df = filtered_df
mc_df['location'] = mc_df['county'] + ", " + mc_df['state']
drop_cols = ['county', 'state']
mc_df = mc_df.drop(columns=drop_cols)
mc_df.set_index(mc_df['location'])

In [ ]:
# Getting average home value for each location
values_df = mc_df.groupby('location', as_index=False)['value'].mean()
values_df = values_df.sort_values(by='value')
list_of_all_counties = values_df['location'].tolist()
list_of_all_counties.sort()
highest_df = values_df.tail(3)
lowest_df = values_df.head(3)
display(highest_df)
display(lowest_df)
#display(list_of_all_counties)

In [ ]:
most_expensive_counties = highest_df['location'].to_numpy()
least_expensive_counties = lowest_df['location'].to_numpy()
display(most_expensive_counties)
display(least_expensive_counties)

In [ ]:
mc_df = mc_df.sort_values(by='date')
expensive_dataframe_array = []
start_date = '2009-04-30'
end_date = '2022-06-30'
for group_loc in most_expensive_counties:
    df_exp_temp = mc_df.loc[(mc_df['location']==group_loc) & (mc_df['date'] <= end_date) & (mc_df['date'] >= start_date)]
    expensive_dataframe_array.append(df_exp_temp.drop('location', axis=1).reset_index())
    
#dataframe_array
expensive_df = pd.concat(expensive_dataframe_array, axis=1, keys=most_expensive_counties)
display(mc_df)
display(expensive_df)

In [ ]:
#mc_df = mc_df.sort_values(by='date')
least_exp_dataframe_array = []
start_date = '2009-04-30'
end_date = '2022-06-30'
for group_loc in least_expensive_counties:
    df_temp = mc_df.loc[(mc_df['location']==group_loc) & (mc_df['date'] <= end_date) & (mc_df['date'] >= start_date)]
    least_exp_dataframe_array.append(df_temp.drop('location', axis=1).reset_index())
    
#dataframe_array
least_exp_df = pd.concat(least_exp_dataframe_array, axis=1, keys=least_expensive_counties)
least_exp_df

In [ ]:
#Monte Carlo Simulation for 3 most expensive counties
mc_expensive_data = MCSimulation(expensive_df, "", 1000, 120)

In [ ]:
display(mc_expensive_data.calc_cumulative_return())

In [ ]:
mc_expensive_data.plot_simulation()

In [ ]:
mc_expensive_data.plot_distribution()

In [ ]:
mc_expensive_data.summarize_cumulative_return()

In [ ]:
#Monte Carlo Simulation for 3 least expensive counties
mc_least_exp_data = MCSimulation(least_exp_df, "", 1000, 120)

In [ ]:
display(mc_least_exp_data.calc_cumulative_return())

In [ ]:
mc_least_exp_data.plot_simulation()

In [ ]:
mc_least_exp_data.plot_distribution()

In [ ]:
mc_least_exp_data.summarize_cumulative_return()